# Library Management System
## Core Code

In [17]:

%%writefile library_management.py
class Library:
    def __init__(self):
        self.books = {}  # {book_id: {'title': str, 'author': str, 'copies': int, 'genre': str}}
        self.members = {}  # {member_id: {'name': str, 'borrowed_books': list}}
        self.borrowed_books = {}  # {book_id: member_id}

    # Book Management
    def add_book(self, book_id, title, author, copies, genre="General"):
        if book_id in self.books:
            self.books[book_id]['copies'] += copies
        else:
            self.books[book_id] = {'title': title, 'author': author, 'copies': copies, 'genre': genre}

    def remove_book(self, book_id):
        if book_id in self.books:
            del self.books[book_id]

    def search_book(self, query, search_by="title"):
        results = [
            book for book in self.books.values()
            if query.lower() in book[search_by].lower()
        ]
        return results

    def list_books_by_genre(self, genre):
        return [book for book in self.books.values() if book['genre'].lower() == genre.lower()]

    def list_available_books(self):
        return {book_id: info for book_id, info in self.books.items() if info['copies'] > 0}

    # Member Management
    def register_member(self, member_id, name):
        if member_id not in self.members:
            self.members[member_id] = {'name': name, 'borrowed_books': []}

    def deregister_member(self, member_id):
        if member_id in self.members:
            del self.members[member_id]

    def get_member_details(self, member_id):
        return self.members.get(member_id, None)

    # Borrowing and Returning Books
    def borrow_book(self, book_id, member_id):
        if book_id in self.books and self.books[book_id]['copies'] > 0:
            if member_id in self.members:
                self.books[book_id]['copies'] -= 1
                self.members[member_id]['borrowed_books'].append(book_id)
                self.borrowed_books[book_id] = member_id
            else:
                raise ValueError("Member ID not found.")
        else:
            raise ValueError("Book unavailable for borrowing.")

    def return_book(self, book_id, member_id):
        if book_id in self.borrowed_books and self.borrowed_books[book_id] == member_id:
            self.books[book_id]['copies'] += 1
            self.members[member_id]['borrowed_books'].remove(book_id)
            del self.borrowed_books[book_id]
        else:
            raise ValueError("Invalid return attempt.")

    # Utility Methods
    def display_books(self):
        if self.books:
            print("Books in the Library:")
            for book_id, info in self.books.items():
                print(
                    f"ID: {book_id}, Title: {info['title']}, Author: {info['author']}, "
                    f"Copies: {info['copies']}, Genre: {info['genre']}"
                )
        else:
            print("No books in the library.")

    def display_members(self):
        if self.members:
            print("Library Members:")
            for member_id, info in self.members.items():
                print(
                    f"ID: {member_id}, Name: {info['name']}, Borrowed Books: {info['borrowed_books']}"
                )
        else:
            print("No members registered.")

    # Additional Features
    def generate_book_report(self):
        report = "Book Report:\n"
        for book_id, info in self.books.items():
            report += (
                f"ID: {book_id}, Title: {info['title']}, Author: {info['author']}, "
                f"Copies: {info['copies']}, Genre: {info['genre']}\n"
            )
        return report

    def generate_member_report(self):
        report = "Member Report:\n"
        for member_id, info in self.members.items():
            report += (
                f"ID: {member_id}, Name: {info['name']}, Borrowed Books: {info['borrowed_books']}\n"
            )
        return report

    def get_most_popular_books(self):
        borrowed_count = {book_id: 0 for book_id in self.books.keys()}
        for book_id in self.borrowed_books.keys():
            borrowed_count[book_id] += 1
        sorted_books = sorted(
            borrowed_count.items(), key=lambda x: x[1], reverse=True
        )
        return sorted_books[:5]


# Sample Usage
if __name__ == "__main__":
    library = Library()

    # Add books
    library.add_book(1, "1984", "George Orwell", 5, genre="Dystopian")
    library.add_book(2, "To Kill a Mockingbird", "Harper Lee", 3, genre="Classic")
    library.add_book(3, "The Great Gatsby", "F. Scott Fitzgerald", 4, genre="Classic")

    # Register members
    library.register_member(101, "Alice")
    library.register_member(102, "Bob")

    # Borrow books
    library.borrow_book(1, 101)
    library.borrow_book(2, 102)

    # Display books and members
    library.display_books()
    library.display_members()

    # Return books
    library.return_book(1, 101)

    # Display updated status
    library.display_books()
    library.display_members()


Overwriting library_management.py


## Test Code

In [18]:
%%writefile test_library_management.py
import unittest
from library_management import Library

class TestLibrary(unittest.TestCase):
    def setUp(self):
        self.library = Library()

    def test_add_book(self):
        self.library.add_book(1, "1984", "George Orwell", 5)
        self.assertIn(1, self.library.books)

    def test_register_member(self):
        self.library.register_member(101, "Alice")
        self.assertIn(101, self.library.members)

    def test_borrow_book(self):
        self.library.add_book(1, "1984", "George Orwell", 5)
        self.library.register_member(101, "Alice")
        self.library.borrow_book(1, 101)
        self.assertEqual(self.library.books[1]['copies'], 4)
        self.assertIn(1, self.library.members[101]['borrowed_books'])

    def test_return_book(self):
        self.library.add_book(1, "1984", "George Orwell", 5)
        self.library.register_member(101, "Alice")
        self.library.borrow_book(1, 101)
        self.library.return_book(1, 101)
        self.assertEqual(self.library.books[1]['copies'], 5)



Overwriting test_library_management.py


### Installing coverage and checking code coverage

In [19]:
!pip install coverage
!coverage run -m unittest library_management.py
!coverage report




----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
Name                    Stmts   Miss  Cover
-------------------------------------------
library_management.py      83     65    22%
-------------------------------------------
TOTAL                      83     65    22%


### Installing mutpy

In [20]:
!pip install mutpy

### Mutation testing

In [21]:
!mut.py --target library_management --unit-test test_library_management

[*] Start mutation process:
   - targets: library_management
   - tests: test_library_management
[*] 4 tests passed:
   - test_library_management [0.00044 s]
[*] Start mutants generation and execution:
   - [#   1] ASR library_management: [0.00580 s] survived
   - [#   2] ASR library_management: [0.00640 s] killed by test_borrow_book (test_library_management.TestLibrary)
   - [#   3] ASR library_management: [0.00648 s] killed by test_return_book (test_library_management.TestLibrary)
   - [#   4] ASR library_management: [0.01254 s] survived
   - [#   5] ASR library_management: [0.01033 s] survived
   - [#   6] ASR library_management: [0.00819 s] survived
   - [#   7] COD library_management: [0.00696 s] killed by test_borrow_book (test_library_management.TestLibrary)
   - [#   8] COI library_management: [0.00681 s] killed by test_add_book (test_library_management.TestLibrary)
   - [#   9] COI library_management: [0.00623 s] killed by test_add_book (test_library_management.TestLibrary)
  